In [1]:
import re
import json
import os
import sys

parent_path = os.path.abspath('..')
sys.path.append(parent_path)
print(parent_path)
parent_path = os.path.abspath('../../')
sys.path.append(parent_path)
print(parent_path)
parent_path = os.path.abspath('../../../')
sys.path.append(parent_path)
print(parent_path)

from dotenv import load_dotenv

# load .env file
project_path = os.path.abspath('../../')
dotenv_path = os.path.join(project_path, '.env')

from dotenv import load_dotenv
load_dotenv(dotenv_path)

print(os.getenv("SISI_DB_TYPE"))

# load project variables
from core.ShoreNet.definitions.variables import VariablesManager

var = VariablesManager(stage_env)
print(var.data_path)


/home/jerry/codebase/sisi/core
/home/jerry/codebase/sisi
/home/jerry/codebase
mysql
/mnt/d/data/sisi


In [17]:
# load dock polygons
from core.ShoreNet.events.generic.tools import load_dock_polygon

dock_polygon_df = load_dock_polygon(var.engine)
print(len(dock_polygon_df), type(dock_polygon_df))

1518 <class 'pandas.core.frame.DataFrame'>


In [3]:
import pandas as pd

from sqlalchemy import text

from core.ShoreNet.events.filter import clean_up_events
from core.ShoreNet.statics.filter import clean_up_statics
from core.ShoreNet.definitions.parameters import TableNames as tbn

# count all sail logging
stop_event_query = f"""
SELECT 
    mmsi, 
    begin_time, end_time, end_time - begin_time as duration, 
    begin_lng, begin_lat, point_num, event_categories, coal_dock_id
FROM 
    sisi.{tbn.all_stop_events_table_name} t
WHERE
    t.avg_speed < 1
"""

coal_static_query = f"""
SELECT
    mmsi, ship_name, ship_type, length, width, dwt
FROM
    sisi.{tbn.dim_ships_statics_table_name}
"""

events_df = pd.read_sql(
    sql=text(stop_event_query), con=var.engine
)

statics_df = pd.read_sql(
    sql=text(coal_static_query), con=var.engine
)
statics_df = clean_up_statics(statics_df)
print(statics_df.shape)

events_df = clean_up_events(
    df=events_df,
    var=var,
    mmsi_enum_list=statics_df['mmsi'].tolist()
)

(11935, 7)
original event data shape: (13642667, 9)
cleaned event data shape: (458276, 9)


In [ ]:
from core.ShoreNet.events.dock import cluster_dock_polygon_dbscan

# find events without polygon and 
# dbscan cluster them
cleaned_evnet_df = cluster_dock_polygon_dbscan(
    events_df=events_df, vars=var
)

# dbscan cluster result
print(f"events cluster count : {cleaned_evnet_df['cluster'].nunique()}")

with_polygon_dbscan_df = cleaned_evnet_df.loc[~cleaned_evnet_df['coal_dock_id'].isna()]
without_polygon_dbscan_df = cleaned_evnet_df.loc[cleaned_evnet_df['coal_dock_id'].isna()]
print(with_polygon_dbscan_df.shape, without_polygon_dbscan_df.shape, cleaned_evnet_df.shape)
print(f"Event with polygon percentage is {100 * with_polygon_dbscan_df.shape[0] / cleaned_evnet_df.shape[0]} %")

events cluster count : 116
(150442, 10) (15457, 10) (165899, 10)
Event with polygon percentage is 90.68288537001429 %


In [15]:
# load beijing demo data
beilun_df = pd.read_excel("/home/jerry/codebase/sisi/data/beijing_demo_data.xlsx", sheet_name="北仑港区")
caofeidian_df = pd.read_excel("/home/jerry/codebase/sisi/data/beijing_demo_data.xlsx", sheet_name="曹妃甸港区")
print(f"beilun mmsi cnt: {beilun_df['mmsi'].nunique()}")
print(f"caofeidian mmsi cnt: {caofeidian_df['mmsi'].nunique()}")
beilun_df.head()

beilun mmsi cnt: 10
caofeidian mmsi cnt: 158


,船名,mmsi,船舶识别号,上一港,下一港,泊位名称,报告时间,进/出港,前吃水,后吃水,货物种类,货运量
0,北仑5,412439190,CN20053162500,京唐,NaN,宁波舟山港北仑港区北司宁钢7#泊位,2023-12-01 07:34:05,进港,6.8,7.0,煤炭,9859
1,北仑5,412439190,CN20053162500,NaN,宁波,宁波舟山港北仑港区北司宁钢7#泊位,2023-12-02 09:34:45,出港,1.8,4.6,煤炭,0
2,北仑7,413416630,CN20079704377,日照,NaN,宁波舟山港北仑港区北司宁钢7#泊位,2023-12-02 14:38:41,进港,6.4,6.6,煤炭,9625
3,北仑7,413416630,CN20079704377,NaN,日照,宁波舟山港北仑港区北司宁钢7#泊位,2023-12-04 11:22:16,出港,2.5,5.1,煤炭,0
4,富兴22,414074000,CN20104564491,日照,NaN,宁波舟山港北仑港区北电码头2#泊位,2023-12-05 21:46:07,进港,12.7,12.7,煤炭,53745


In [72]:
from tqdm import tqdm

def match_dock_details(dock_id, con):
    df = pd.read_sql(
        sql=f"""
        SELECT
            Id as dock_id,
            Name as name,
            Province as province,
            Distruct as distruct
        FROM
            sisi.dim_dock_polygon
        WHERE
            Id = {dock_id} 
        """,
        con=con
    )
    if df.shape[0] > 0:
        return df.iloc[0]
    else:
        return None

start_ts = 1698595200

cfd_appearing_cnt = 0
observe_mmsi_cnt = 0
observe_mmsi_ls = caofeidian_df["mmsi"].unique().tolist()
unobserve_mmsi_ls = []
mmsi_sailing_chain_dict = {}
for _mmsi in tqdm(observe_mmsi_ls):
    # obverse_df = caofeidian_df.loc[beilun_df["mmsi"]==_mmsi]
    
    # load mmsi od pairs from mysql
    _mmsi_pair_df = pd.read_sql(
        sql=f"""
        SELECT 
            departure_dock_id, arrival_dock_id 
        FROM sisi.data_od_pairs 
        WHERE 
            mmsi = {_mmsi}
            AND arrival_time > {start_ts}
        """,
        con=var.engine
    )
    if _mmsi_pair_df.shape[0] == 0:
        unobserve_mmsi_ls.append(_mmsi)
        continue
    
    # print(f"mmsi -> {_mmsi}, od pairs cnt -> {_mmsi_pair_df.shape[0]}")
    distruct_ls = []
    direction_symbol = " ==> "
    for _, row in _mmsi_pair_df.iterrows():
        departure_dock_details = match_dock_details(row["departure_dock_id"], var.engine)
        arrival_dock_details = match_dock_details(row["arrival_dock_id"], var.engine)
        departure_region_str = f"{departure_dock_details['province']}-{departure_dock_details['distruct']}"
        arrvial_region_str = f"{arrival_dock_details['province']}-{arrival_dock_details['distruct']}"
        distruct_ls.append(f"{departure_region_str}{direction_symbol}{arrvial_region_str}")

    sailing_chain = direction_symbol.join(distruct_ls)
    mmsi_sailing_chain_dict[_mmsi] = sailing_chain
    if "曹妃甸" in sailing_chain:
        cfd_appearing_cnt += 1
        
    observe_mmsi_cnt += 1

print(f"per -> {cfd_appearing_cnt} / {len(caofeidian_df['mmsi'].unique().tolist())}")
print(f"observe per -> {cfd_appearing_cnt} / {observe_mmsi_cnt}")
    

100%|██████████| 158/158 [01:05<00:00,  2.41it/s]

per -> 29 / 158
observe per -> 29 / 62


In [79]:
from pprint import pprint

def match_static_details(mmsi, con):
    df = pd.read_sql(
        sql=f"""select * from sisi.dim_ships_statics where mmsi = {mmsi}""",
        con=con
    )
    return df


def match_event_details(mmsi, con):
    df = pd.read_sql(
        sql=f"""select * from sisi.factor_all_stop_events where mmsi = {mmsi} and begin_time > {start_ts}""",
        con=con
    )
    return df

lack_static_mmsi_ls = []
lack_events_mmsi_ls = []
lack_dock_id_mmsi_ls = []
for _mmsi in unobserve_mmsi_ls:
    _mmsi_static_details = match_static_details(_mmsi, var.engine)
    if _mmsi_static_details.shape[0] == 0:
        lack_static_mmsi_ls.append(_mmsi)
        continue
    
    # check if dock_id is NULL in event table
    _mmsi_event_details = match_event_details(_mmsi, var.engine)
    if _mmsi_event_details.shape[0] == 0:
        lack_events_mmsi_ls.append(_mmsi)
        continue
    
    coal_dock_id_na_ls = _mmsi_event_details.loc[_mmsi_event_details["coal_dock_id"].isna()]["coal_dock_id"].tolist()
    if len(coal_dock_id_na_ls) == _mmsi_event_details.shape[0]:
        lack_dock_id_mmsi_ls.append(_mmsi)
        continue
    
    print(_mmsi)
    
len(unobserve_mmsi_ls), len(lack_static_mmsi_ls), len(lack_events_mmsi_ls), len(lack_dock_id_mmsi_ls)

413345750
414640000
412473010
414006000
413130000
414402620
413448950
414557000
414847000
413415250


(96, 50, 3, 33)